## The output of the code has been redacted in order to preserve paritipant data. What is observed in this notebook can be viewed in the associated paper which is being prepared for publication

In [ ]:
# Import Python Packages
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
import seaborn as sns
from scipy.stats import shapiro
from scipy.stats import spearmanr
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy.stats.mstats import winsorize
from scipy.stats import chi2
import statsmodels.api as sm
from scipy.stats import kstest
from scipy.stats import wilcoxon
import statsmodels.formula.api as smf
from scipy.stats import levene
import statsmodels.api as sm

In [ ]:
#Load in Data
os.chdir('/Users/[REDACTED]/Desktop/tACS_Pilot_Study')
df = pd.read_spss('tACS_Reanalysis_20220208_noSevereBAI.sav')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
display(df)

Missing Data Analysis 
Visual inspection shows N=8, only one participant is sowing NaNs. Column wise, this means that there is 12.5%
Missing data - This means that data imputation would not be feasible. 

HOWEVER, a recent paper by Madley-Dowd et al. (2019) showed MI is still valid up to 90% missing data so MI will be used to impute the missing FAA values. 

In [ ]:
# Extract relevant Data for graphs
df2 = df[['Day1PeakAsym','Day3PeakAsym','Day1PreSAS','Day3PostSAS','Day1PreBAI','Day3PostBAI','Day1PreBDI',
          'Day3PostBDI','Day1PrePANAS_P','Day3PostPANAS_P','Day1PrePANAS_N','Day3PostPANAS_N']]
display(df2)

In [ ]:
#Multiple Imputation
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(df2)
IterativeImputer(random_state=0)
X_test = df2
# the model learns that the second feature is double the first
display(imp.transform(X_test))

Multiple imputation has provided the first Day1PeakAsym as [REDACTED] and the Day3PeakAsym as [REDACTED]. 

The df.replace() function wasn't working so I manually imported it into the dataset

In [ ]:
df3 = pd.read_csv('tACS_Reanalysis_20220208_noSevereBAI.csv')
df3 = df3[['Day1PeakAsym','Day3PeakAsym','Day1PreSAS','Day3PostSAS','Day1PreBAI','Day3PostBAI','Day1PreBDI',
          'Day3PostBDI','Day1PrePANAS_P','Day3PostPANAS_P','Day1PrePANAS_N','Day3PostPANAS_N']]
display(df3)

Now that multiple imputation has been completed. We can move on to the assessment of the following:
Outliers (Univariate and multivariate)
Normality
This will dictate the type of correlation that will be used - because the sample size is small it'll likely that spearman's will be used anyway. 

In [ ]:
sns.set(rc={"figure.figsize":(25, 10)})
sns.boxplot(data = df3)

As can be seen here: There are 2 outliers in the Day1PeakAsym and an outlier in the Day3PostBDI. 
To mitigate these outliers, I prefer to winsorize the data. As the датасет is very small, it's better to winzorise than to remove outliers. Preserving data as much as possible. 

In [ ]:
#Create variable with Day1PeakAsym winsorized. 
df3['Day1PeakAsym_win'] = winsorize(df3['Day1PeakAsym'], (0.3, 0.1))
display(df3)
sns.boxplot(data = df3)

In [ ]:
#Create variable with Day3PostBAI winsorized. 
df3['Day3PostBAI_win'] = winsorize(df3['Day3PostBAI'], (0.1, 0.2))
display(df3)
sns.boxplot(data = df3)

Remove the variables that are now redundant: These are the Day1PeakAsym & Day3PostBAI

In [ ]:
df4 = df3[['Day1PeakAsym_win','Day3PeakAsym','Day1PreSAS','Day3PostSAS','Day1PreBAI','Day3PostBAI_win','Day1PreBDI',
          'Day3PostBDI','Day1PrePANAS_P','Day3PostPANAS_P','Day1PrePANAS_N','Day3PostPANAS_N']]
display(df4)

Now we have this done, we can identify multivariate outliers using mahalanobis distance

In [ ]:
def mahalanobis(x=None, data=None, cov=None):

    x_mu = x - np.mean(df4)
    if not cov:
        cov = np.cov(df4.values.T)
    inv_covmat = np.linalg.inv(cov)
    left = np.dot(x_mu, inv_covmat)
    mahal = np.dot(left, x_mu.T)
    return mahal.diagonal()

In [ ]:
df4['mahalanobis'] = mahalanobis(x=df4, data=df4[['Day1PeakAsym_win', 'Day3PeakAsym', 'Day1PreSAS', 'Day3PostSAS', 
                                                 'Day3PostSAS', 'Day1PreBAI', 'Day3PostBAI_win', 'Day1PreBDI',
                                                'Day3PostBDI', 'Day1PrePANAS_P', 'Day3PostPANAS_P', 'Day1PrePANAS_N', 
                                                 'Day3PostPANAS_N']])
display(df4)

In [ ]:
#Calculate Mahalanobis p value
df4['p'] = 1 - chi2.cdf(df4['mahalanobis'], 11)
display(df4)

Mahalanobis distance df is N-1. The p values show there are no multivariate outliers. The threshold is typically p <.001.

The next thing to assess is normality. The sample size is small so non-parametric testing will be used anyway, it's worth checking nonetheless. the boxplots also suggest most variables will be non-normal.

For normality. The Kolmogorov-Smirnov test was used because it is a nonparametric test and with small sample sizes like this, probably the better test. 

In [ ]:
plt.hist(df4['Day1PeakAsym_win'])
sm.qqplot(df4['Day1PeakAsym_win'], line='45')
kstest(df4['Day1PeakAsym_win'], 'norm')

The graphs show large deviation from normality, further confirmed by the Kolmogorov-Smirnov test statistic. Shapiro-Wilk was run which suggested from the analysis that the data was normally distributed, thus also confirming the use of Kolmogorov-Smirnov over Shapiro-Wilk

In [ ]:
plt.hist(df4['Day3PeakAsym'])
sm.qqplot(df4['Day3PeakAsym'], line='45')
kstest(df4['Day3PeakAsym'], 'norm')

In [ ]:
plt.hist(df4['Day1PreSAS'])
sm.qqplot(df4['Day1PreSAS'], line='45')
kstest(df4['Day1PreSAS'], 'norm')

In [ ]:
plt.hist(df4['Day3PostSAS'])
sm.qqplot(df4['Day3PostSAS'], line='45')
kstest(df4['Day3PostSAS'], 'norm')

In [ ]:
plt.hist(df4['Day1PreBAI'])
sm.qqplot(df4['Day1PreBAI'], line='45')
kstest(df4['Day1PreBAI'], 'norm')

In [ ]:
plt.hist(df4['Day3PostBAI_win'])
sm.qqplot(df4['Day3PostBAI_win'], line='45')
kstest(df4['Day3PostBAI_win'], 'norm')

In [ ]:
plt.hist(df4['Day1PreBDI'])
sm.qqplot(df4['Day1PreBDI'], line='45')
kstest(df4['Day1PreBDI'], 'norm')

In [ ]:
plt.hist(df4['Day3PostBDI'])
sm.qqplot(df4['Day3PostBDI'], line='45')
kstest(df4['Day3PostBDI'], 'norm')

In [ ]:
plt.hist(df4['Day1PrePANAS_P'])
sm.qqplot(df4['Day1PrePANAS_P'], line='45')
kstest(df4['Day1PrePANAS_P'], 'norm')

In [ ]:
plt.hist(df4['Day3PostPANAS_P'])
sm.qqplot(df4['Day3PostPANAS_P'], line='45')
kstest(df4['Day3PostPANAS_P'], 'norm')

In [ ]:
plt.hist(df4['Day1PrePANAS_N'])
sm.qqplot(df4['Day1PrePANAS_N'], line='45')
kstest(df4['Day1PeakAsym_win'], 'norm')

In [ ]:
plt.hist(df4['Day3PostPANAS_N'])
sm.qqplot(df4['Day3PostPANAS_N'], line='45')
kstest(df4['Day3PostPANAS_N'], 'norm')

Across all variables, p values varied, histograms and QQ plots suggested deviations from normality across all variables. Boxplots also confirmed that none of these variables are normally distributed. 8 cases is likely to mean the p value for normality tests are unstable.

In [ ]:
#Linearity
pd.plotting.scatter_matrix(df4[['Day1PeakAsym_win', 'Day3PeakAsym', 'Day1PreSAS', 'Day3PostSAS', 'Day1PreBAI', 'Day3PostBAI_win', 'Day1PreBDI',
            'Day3PostBDI', 'Day1PrePANAS_P', 'Day3PostPANAS_P', 'Day1PrePANAS_N', 'Day3PostPANAS_N']],
                           alpha = 1, figsize = [25, 25]);

In [ ]:
spearmancorr1 = spearmanr(df4[['Day1PeakAsym_win', 'Day3PeakAsym', 'Day1PreSAS', 'Day3PostSAS', 'Day1PreBAI', 'Day3PostBAI_win', 'Day1PreBDI',
            'Day3PostBDI', 'Day1PrePANAS_P', 'Day3PostPANAS_P', 'Day1PrePANAS_N', 'Day3PostPANAS_N']], axis = 0)
display(spearmancorr1)

Create dataframes to display these results in a slightly more paletable way: 

In [ ]:
variables = ([['Day1PeakAsym_win', 'Day3PeakAsym', 'Day1PreSAS', 'Day3PostSAS', 'Day1PreBAI', 'Day3PostBAI_win', 'Day1PreBDI',
            'Day3PostBDI', 'Day1PrePANAS_P', 'Day3PostPANAS_P', 'Day1PrePANAS_N', 'Day3PostPANAS_N']])

corvals = pd.DataFrame([[1.        ,  0.15256429, -0.61025715, -0.30770355,  0.11915865,
        -0.16309163, -0.29789662, -0.41705526,  0.37293493,  0.40854393,
        -0.45769286, -0.08040844],
       [ 0.15256429,  1.        , -0.55      , -0.80675117, -0.62762056,
        -0.88614695, -0.52720127, -0.42678198,  0.5       ,  0.76151294,
        -0.58333333, -0.25473903],
       [-0.61025715, -0.55      ,  1.        ,  0.72271459,  0.4853599 ,
         0.23630585,  0.6443571 ,  0.55230609, -0.35      , -0.37657233,
         0.6       ,  0.07905694],
       [-0.30770355, -0.80675117,  0.72271459,  1.        ,  0.43882247,
         0.64255901,  0.49789473,  0.75528098, -0.63867801, -0.75950043,
         0.53783412,  0.56692694],
       [ 0.11915865, -0.62762056,  0.4853599 ,  0.43882247,  1.        ,
         0.30085826,  0.82773109, -0.05462185, -0.20920685, -0.09243697,
         0.69456675, -0.22934432],
       [-0.16309163, -0.88614695,  0.23630585,  0.64255901,  0.30085826,
         1.        ,  0.23305922,  0.42798147, -0.59920413, -0.89833734,
         0.42197474,  0.30691229],
       [-0.29789662, -0.52720127,  0.6443571 ,  0.49789473,  0.82773109,
         0.23305922,  1.        ,  0.23109244, -0.41004543, -0.21848739,
         0.91214188, -0.07938842],
       [-0.41705526, -0.42678198,  0.55230609,  0.75528098, -0.05462185,
         0.42798147,  0.23109244,  1.        , -0.7113033 , -0.74369748,
         0.32636269,  0.55571893],
       [ 0.37293493,  0.5       , -0.35      , -0.63867801, -0.20920685,
        -0.59920413, -0.41004543, -0.7113033 ,  1.        ,  0.78661776,
        -0.66666667, -0.36893239],
       [ 0.40854393,  0.76151294, -0.37657233, -0.75950043, -0.09243697,
        -0.89833734, -0.21848739, -0.74369748,  0.78661776,  1.        ,
        -0.44351853, -0.48515145],
       [-0.45769286, -0.58333333,  0.6       ,  0.53783412,  0.69456675,
         0.42197474,  0.91214188,  0.32636269, -0.66666667, -0.44351853,
         1.        ,  0.04392052],
       [-0.08040844, -0.25473903,  0.07905694,  0.56692694, -0.22934432,
         0.30691229, -0.07938842,  0.55571893, -0.36893239, -0.48515145,
         0.04392052,  1.        ]], variables, variables)

pvals = pd.DataFrame([[0.00000000e+00, 6.95164092e-01, 8.09396701e-02, 4.20527287e-01,
        7.60105950e-01, 6.75029099e-01, 4.36236135e-01, 2.64094827e-01,
        3.22898563e-01, 2.74963884e-01, 2.15411703e-01, 8.37073284e-01],
       [6.95164092e-01, 0.00000000e+00, 1.24976784e-01, 8.59808327e-03,
        7.03650228e-02, 1.46441911e-03, 1.44699332e-01, 2.51956690e-01,
        1.70470661e-01, 1.71171255e-02, 9.91858165e-02, 5.08299873e-01],
       [8.09396701e-02, 1.24976784e-01, 0.00000000e+00, 2.78302399e-02,
        1.85353818e-01, 5.40443782e-01, 6.10329577e-02, 1.23076630e-01,
        3.55819573e-01, 3.17822682e-01, 8.76228290e-02, 8.39782984e-01],
       [4.20527287e-01, 8.59808327e-03, 2.78302399e-02, 0.00000000e+00,
        2.37352707e-01, 6.19957144e-02, 1.72567247e-01, 1.86107044e-02,
        6.41062083e-02, 1.75903842e-02, 1.35288910e-01, 1.11431747e-01],
       [7.60105950e-01, 7.03650228e-02, 1.85353818e-01, 2.37352707e-01,
        0.00000000e+00, 4.31465115e-01, 5.87748490e-03, 8.89001573e-01,
        5.89047241e-01, 8.13024659e-01, 3.78640538e-02, 5.52781099e-01],
       [6.75029099e-01, 1.46441911e-03, 5.40443782e-01, 6.19957144e-02,
        4.31465115e-01, 4.74758611e-56, 5.46184354e-01, 2.50480845e-01,
        8.81551018e-02, 9.97418051e-04, 2.57917859e-01, 4.21785170e-01],
       [4.36236135e-01, 1.44699332e-01, 6.10329577e-02, 1.72567247e-01,
        5.87748490e-03, 5.46184354e-01, 0.00000000e+00, 5.49673144e-01,
        2.73029747e-01, 5.72229255e-01, 6.06850679e-04, 8.39118250e-01],
       [2.64094827e-01, 2.51956690e-01, 1.23076630e-01, 1.86107044e-02,
        8.89001573e-01, 2.50480845e-01, 5.49673144e-01, 0.00000000e+00,
        3.16565570e-02, 2.16118918e-02, 3.91360449e-01, 1.20296343e-01],
       [3.22898563e-01, 1.70470661e-01, 3.55819573e-01, 6.41062083e-02,
        5.89047241e-01, 8.81551018e-02, 2.73029747e-01, 3.16565570e-02,
        0.00000000e+00, 1.19092120e-02, 4.98672306e-02, 3.28530352e-01],
       [2.74963884e-01, 1.71171255e-02, 3.17822682e-01, 1.75903842e-02,
        8.13024659e-01, 9.97418051e-04, 5.72229255e-01, 2.16118918e-02,
        1.19092120e-02, 0.00000000e+00, 2.31784065e-01, 1.85570845e-01],
       [2.15411703e-01, 9.91858165e-02, 8.76228290e-02, 1.35288910e-01,
        3.78640538e-02, 2.57917859e-01, 6.06850679e-04, 3.91360449e-01,
        4.98672306e-02, 2.31784065e-01, 0.00000000e+00, 9.10669551e-01],
       [8.37073284e-01, 5.08299873e-01, 8.39782984e-01, 1.11431747e-01,
        5.52781099e-01, 4.21785170e-01, 8.39118250e-01, 1.20296343e-01,
        3.28530352e-01, 1.85570845e-01, 9.10669551e-01, 0.00000000e+00]], variables, variables)

In [ ]:
display(corvals)
# Correlaton coefficients

In [ ]:
display(pvals)
# P values

In [ ]:
#Heatmaps to make correlations easier to visualise
sns.heatmap(corvals, annot = True)

In [ ]:
sns.heatmap(pvals, annot = True)

The correlations show the following:
significant correlations for 

Day3PostSAS & Day3PeakAsym

Day3PostBAI_win & Day3PostPANAS_P

Day3PostSAS & Day1PreSAS

Day3PostBDI & Day3PostSAS

Day3PostPANAS_P & Day3PostSAS 

Day1PreBDI & Day1PreBAI

Day3PostPANAS_P & Day3PostBAI_win

Day1PrePANAS_N & Day1PreBDI

Day1PrePANAS_P & Day3PostBDI

Day3PostPANAS_P & Day3PostBDI

Day3PostPANAS_P & Day1PrePANAS_P

Day1PrePANAS_N & Day1PrePANAS_P

Wilcoxon Signed Ranks Test

In [ ]:
wilcoxon(df4['Day1PeakAsym_win'], df4['Day3PeakAsym'], mode = 'approx')

In [ ]:
wilcoxon(df4['Day1PreSAS'], df4['Day3PostSAS'], mode = 'approx')

In [ ]:
wilcoxon(df4['Day1PreBAI'], df4['Day3PostBAI_win'], mode = 'approx')

In [ ]:
wilcoxon(df4['Day1PreBDI'], df4['Day3PostBDI'], mode = 'approx')

In [ ]:
wilcoxon(df4['Day1PrePANAS_P'], df4['Day3PostPANAS_P'], mode = 'approx')

In [ ]:
wilcoxon(df4['Day1PrePANAS_N'], df4['Day3PostPANAS_N'], mode = 'approx')

In [ ]:
np.mean(df4)

Basically; tACS at 10Hz improved SAS scores, BAI scores, BDI scores, and PANAS_N scores. 

Observations of the means suggested:

SAS went down, BAI went down, BDI went down and PANAS_N went down.

# Regression

Assumptions:

Normality of the DV - however using an OLS model reduces the error variance of the summs of squares thus making normality a non-issue.
 
Homoscedasticity - This will be done by using Levene's test. If the data is normally distributed you can use Bartlett's test

Independence - don't do regressions with day1 and day3 of the same measure

Linearity - Linear relationships only - not enough cases to establish visually, best way to assess is by using significant correlations. 

Multicollinearity/Singularity - corrcoefs >.8 will need to be excluded. You can average collinear variables. 

Collinear Variables:
- Day1PreBDI and Day1PreBAI
- Day1PrePANAS_N and Day1PreBDI

Homoscedasticity via levene's test. 

Multiple regressions showed no results of interest. Likely because a MR requires 20N per predictor variable - and we're using 9

# Simple Linear Regressions

For the assumption of Homoscedasticity: The following regression models would have met the assumption:

In [ ]:
#Levene's Test for Homoscedasticity
stats.levene(df4['Day1PeakAsym_win'], df4['Day1PreSAS'], center = 'mean')

In [ ]:
stats.levene(df4['Day3PeakAsym'], df4['Day3PostSAS'], center = 'mean')

In [ ]:
reg1 = smf.ols('Day3PostSAS~Day3PeakAsym', data=df4).fit()
reg2 = smf.ols('Day1PreSAS~Day1PeakAsym_win', data=df4).fit()

In [ ]:
display(reg1.summary())

fig_reg1 = plt.figure(figsize=(12,8))
fig_reg1 = sm.graphics.plot_regress_exog(reg1, 'Day3PeakAsym', fig = fig_reg1)

In [ ]:
display(reg2.summary())

fig_reg2 = plt.figure(figsize=(12,8))
fig_reg2 = sm.graphics.plot_regress_exog(reg2, 'Day1PeakAsym_win', fig = fig_reg2)

The issue with these regression models is that although levene's test was non-significant for both models, the residual plots show coning suggesting that the data is not homoscedasticstic. Due to how many participants are in the sample, I'd trust the plot over the test. Therefore, the assumption of Homoscedasticity is breached. Observations of Residuals show evidence of coning in both regression models so the p values are likely invalidated. This is further supported by the lack of power and participants

# Graphs

## Bar Graph showing pre & post changes

In [ ]:
plt.savefig('bargraph.jpg')  
plt.rc('font', size=15)

Day1 = [-2.51, 34.8, 14.7, 14.8, 29.2, 24.0]
Day3 = [-0.26, 28.1, 3.33, 7.33, 30.4, 13.7]
Day1err = [1.76, 1.75, 2.77, 2.56, 2.94, 2.28] 
Day3err = [1.40, 1.32, 1.26, 1.30, 3.47, 0.86]

x = ['FAA', 'SAS', 'BAI', 'BDI', 'PANAS(P)', 'PANAS(N)']
lgnd = ['Day 1', 'Day 3']
f = plt.figure()
f.set_figwidth(10)
f.set_figheight(10)

width =0.3
plt.bar(x, Day1, width=width)
plt.bar(np.arange(len(Day3)) + width, Day3, width=width)
plt.legend(lgnd,loc=1)
plt.errorbar(x, Day1, yerr=Day1err, color = 'black', linestyle = '', capsize = 5)
plt.errorbar(np.arange(len(Day3))+ width, Day3, yerr=Day3err, color = 'black', linestyle = '', 
             capsize = 5)
#plt.title('Changes in Self Reported Measures of, Depression, Anxiety and 
            #\n Affect between Day 1 and Day 3. \n', weight='bold')
plt.xlabel('measure')
plt.ylabel('score')
plt.show() 

## Scatter Plot for FAA and BAI at Day 1 and Day 3

Worth noting, the points on the graph are not even, because 2 of the blue and orange dots have the exact same coordinates.

In [ ]:
f = plt.figure()
f.set_figwidth(8)
f.set_figheight(8)
plt.title('Scatterplot of Peak Asymmetry and SAS \n at Day 1 and Day 3 \n', weight='bold')
plt.xlabel('\n Peak Asymmetry \n')
plt.ylabel('SAS \n')
line1, = plt.plot(1, marker='o', label='Day 1')
line2, = plt.plot(1, marker='o', label='Day 3')
plt.legend()
plt.xlim(-7, 6)
plt.ylim(20, 45)
plt.scatter(df4['Day1PeakAsym_win'], df4['Day1PreSAS'])
plt.scatter(df4['Day3PeakAsym'], df4['Day3PostSAS'])

Frontal Alpha Asymmetry (FAA) as a predictor of Anxeity and the efficacy of tACS as a means to reduce this.

The predictive utility of Frontal Alpha Asymmetry on Anxiety and the remedial effect of tACS on anxiety, depression and affect.